In [85]:
"""
Author
Johan Tejning  - jote16@student.bth.se
Tokunbo Ojo    -  tooj21@student.bth.se
"""

'\nAuthor\nJohan Tejning  - jote16@student.bth.se\nTokunbo Ojo    -  tooj21@student.bth.se\n'

In [86]:
#Import libraries
from scipy import stats
from scipy.stats import studentized_range
import pandas as pd
import numpy as np
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import scipy.stats
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import time
import math

In [87]:
#Read spambase.data into pandas dataframe
df = pd.read_csv('spambase.data',header = None)
df

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.000,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.010,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4596,0.31,0.00,0.62,0.0,0.00,0.31,0.00,0.00,0.00,0.00,...,0.000,0.232,0.0,0.000,0.000,0.000,1.142,3,88,0
4597,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.353,0.000,0.000,1.555,4,14,0
4598,0.30,0.00,0.30,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.102,0.718,0.0,0.000,0.000,0.000,1.404,6,118,0
4599,0.96,0.00,0.00,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.057,0.0,0.000,0.000,0.000,1.147,5,78,0


In [88]:
# split time serie into train and test
x = df.iloc[:, 0:57]
y = df[57]
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [89]:
# create discretizer for Equal Width Discretizer
kbins = KBinsDiscretizer(n_bins=2, strategy='uniform', encode='ordinal')

In [90]:
# Equal-Width Discretization implementation
def descretize(data):
    frames = []
    for i in range(0,57):
        bin = kbins.fit_transform(np.array(data.iloc[:, i:i + 1]).reshape(-1,1))
        frames.append(pd.DataFrame(bin, columns=[str(i)]))
    return pd.concat(frames, axis=1)
X_train = descretize(X_train)
X_test = descretize(X_test)
X_train

,0,1,2,3,4,5,6,7,8,9,...,47,48,49,50,51,52,53,54,55,56
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3675,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3676,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3677,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3678,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [91]:
names = [
    "Nearest Neighbors",
    "Linear SVM",
   # "Decision Tree",
   # "Random Forest",
    "Naive Bayes"
]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
  #  DecisionTreeClassifier(max_depth=5),
  #  RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    GaussianNB()
]

In [ ]:
# iterate over classifiers
fold = KFold(n_splits=10, random_state=1, shuffle=True)
performance = []
accuracy = []
f_measure = []
evaluation_measure = {}
cross_validation_result = {}
for name, clf in zip(names, classifiers):
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    performance.append(end - start)
    evaluation_measure["Training Time"] = performance
    #score = clf.score(X_test, y_test)
    y_pred = clf.predict(X_test) # make predictions on test set
    acc = accuracy_score(y_test.values, y_pred)
    accuracy.append(acc)
    evaluation_measure["Accuracy"] = accuracy
    f_score = f1_score(y_test.values, y_pred)
    f_measure.append(f_score)
    evaluation_measure["F Measure"] = f_measure
    cross_validation_result[name] = cross_val_score(clf, x, y, scoring='accuracy', cv=fold , n_jobs=-1)


In [ ]:
#1) computational performance in terms of training time,
performance

In [ ]:
#) predictive performance based on accuracy,
accuracy

In [ ]:
#3) predictive performance based on F-measure.
f_measure

In [ ]:
#The performance, accuracy and f_measure in tabular form
pd.DataFrame.from_dict(evaluation_measure, orient='index',
                       columns=names)

In [ ]:
#Run stratified ten-fold cross-validation tests.
cross_validation_result

In [ ]:
#Cross Validation result in tabular form
cross_validation_df = pd.DataFrame.from_dict(cross_validation_result)
cross_validation_df

In [ ]:
#Calculating Friedman test
#We first need to calculate the ranks
rank1 = []
rank2 = []
rank3 = []
for index, row in cross_validation_df.iterrows():
    temp = row.copy()
    sorted = np.sort(temp.values)
    rank1.append(sorted.tolist().index(row[0]) + 1)
    rank2.append(sorted.tolist().index(row[1]) + 1)
    rank3.append(sorted.tolist().index(row[2]) + 1)

In [ ]:
#Displaying the ranks
result = {}
result["Nearest Neighbors"] = rank1
result["Linear SVM"] = rank2
result["Naive Bayes"] = rank3
ranking_df = pd.DataFrame.from_dict(result)
ranking_df

In [ ]:
rank_avg = ranking_df.mean(axis=0)
rank_avg

In [ ]:
#Finding the sum of squared differences 2
sum = 0
for x in rank_avg.values:
    sum = sum + (x - 2) ** 2
sum_of_squared_differences_1 = 10 * sum
sum_of_squared_differences_1

In [ ]:
#Finding the sum of squared differences 2

summation = 0
for index, row in ranking_df.iterrows():
    summation = summation + ((row[0] - 2) ** 2) + ((row[1] - 2) ** 2) + ((row[2] - 2) ** 2)
sum_of_squared_differences_2 = summation * 0.05
sum_of_squared_differences_2

In [ ]:
#Finding p-value
p_value = scipy.stats.chi2.ppf(0.95, 3)
p_value

In [ ]:
friedman_statistic = sum_of_squared_differences_1 / sum_of_squared_differences_2
decision = "greater than " if friedman_statistic > p_value else "less than "
print("Since the Friedman statistic " + str(friedman_statistic) + " is " + decision + str(p_value) + " so we cannot reject the null hypothesis that all algorithms perform equally")

In [ ]:
#Finding critical difference (CD) using Nemenyi test
#The q alpha 2.343 is taken from the course textbook and also from this link https://plos.figshare.com/articles/dataset/Critical_values_for_the_two-tailed_Nemenyi_test_/7299293/1
#k = 3, n =  10

cd = 2.343 * math.sqrt(3 * (3 + 1) / (6 * 10))
cd

In [ ]:
#Nearest Neighbors - rank_avg[0]
#Linear SVM - rank_avg[1]
#Naive Bayes - rank_avg[2]
no_of_means__of_ranks = len(rank_avg)
for i in range(no_of_means__of_ranks - 1):
    for j in range(i + 1, no_of_means__of_ranks):
        diff = rank_avg[i] - rank_avg[j]
        decision = " and it exceeds the critical difference " if diff > cd else " and does not exceeds the critical difference "
        print("The difference of " + rank_avg.index[i] + " - " + str(rank_avg[i])  + " and " + rank_avg.index[j] + " - " + str(rank_avg[i]) + " is " + str(abs(diff)) +  decision + str(cd))
